## EDA with Red Resort Snowfall Data

__Objectives:__
1. Investigate which week and two week periods have the highest average snowfall.
2. Determine relationship between El Niño-Southern Oscillation (ENSO) and total snowfall over period November - April.


### Objective 1: Relationship Between ENSO and Total Snowfall

In [21]:
# Import Required Libraries

import requests
import pandas as pd
import math


In [7]:
# Pull Red Resort JSON Files

BASE_URL = "https://api.redresort.com/report/history"
YEARS = range(2004, 2024)  # inclusive of 2023

def fetch_year(year: int) -> pd.DataFrame:
    url = f"{BASE_URL}?year={year}"
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()
    # If the API returns a list of records, json_normalize will convert to rows.
    # Adjust if the structure differs.
    return pd.json_normalize(data).assign(year=year)

frames = []
for y in YEARS:
    try:
        frames.append(fetch_year(y))
    except Exception as exc:
        print(f"Failed for {y}: {exc}")

df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

df.head()



,WeatherData,RequestedYear,MountainReport.MRID,MountainReport.SkyB,MountainReport.SkyBText,MountainReport.SkyS,MountainReport.SkySText,MountainReport.MetricToday,MountainReport.MetricOvernight,MountainReport.Metric24hours,...,MountainReport.Notes,MountainReport.SummerMode,MountainReport.DateTime,MountainReport.MailingListID,MountainReport.UserID,MountainReport.Lifts,MountainReport.SnowData,MountainReport.HeaderImage,MountainReport.TotalSnowDepth,year
0,None,2004,0,0,,0,,0,0,0,...,,False,0001-01-01T00:00:00Z,0,0,None,"[{'Month': 'October', 'Year': 2004, 'Total': 4...",,487,2004
1,None,2005,0,0,,0,,0,0,0,...,,False,0001-01-01T00:00:00Z,0,0,None,"[{'Month': 'October', 'Year': 2005, 'Total': 2...",,781,2005
2,None,2006,0,0,,0,,0,0,0,...,,False,0001-01-01T00:00:00Z,0,0,None,"[{'Month': 'October', 'Year': 2006, 'Total': 0...",,434,2006
3,None,2007,0,0,,0,,0,0,0,...,,False,0001-01-01T00:00:00Z,0,0,None,"[{'Month': 'October', 'Year': 2007, 'Total': 0...",,577,2007
4,None,2008,0,0,,0,,0,0,0,...,,False,0001-01-01T00:00:00Z,0,0,None,"[{'Month': 'October', 'Year': 2008, 'Total': 0...",,469,2008


In [10]:
### Remove cols not required from Data Frame

# columns to keep
cols_keep = ["MountainReport.SnowData", "MountainReport.TotalSnowDepth", "year" ]

# select only those columns that exist; drops all others
df = df.loc[:, df.columns.intersection(cols_keep)]

# quick check
df.head()


,MountainReport.SnowData,MountainReport.TotalSnowDepth,year
0,"[{'Month': 'October', 'Year': 2004, 'Total': 4...",487,2004
1,"[{'Month': 'October', 'Year': 2005, 'Total': 2...",781,2005
2,"[{'Month': 'October', 'Year': 2006, 'Total': 0...",434,2006
3,"[{'Month': 'October', 'Year': 2007, 'Total': 0...",577,2007
4,"[{'Month': 'October', 'Year': 2008, 'Total': 0...",469,2008


In [16]:
### Split Data Frame into Season Totals and Daily Snowfall

# columns to keep
daily_cols_keep = ["MountainReport.SnowData"]

# select only those columns that exist; drops all others
df_daily = df.loc[:, df.columns.intersection(daily_cols_keep)]

# columns to keep
totals_cols_keep = ["MountainReport.TotalSnowDepth", "year" ]

# select only those columns that exist; drops all others
df_totals = df.loc[:, df.columns.intersection(totals_cols_keep)]

In [17]:
df_daily.head()

,MountainReport.SnowData
0,"[{'Month': 'October', 'Year': 2004, 'Total': 4..."
1,"[{'Month': 'October', 'Year': 2005, 'Total': 2..."
2,"[{'Month': 'October', 'Year': 2006, 'Total': 0..."
3,"[{'Month': 'October', 'Year': 2007, 'Total': 0..."
4,"[{'Month': 'October', 'Year': 2008, 'Total': 0..."


In [18]:
df_totals.head()

,MountainReport.TotalSnowDepth,year
0,487,2004
1,781,2005
2,434,2006
3,577,2007
4,469,2008


In [ ]:
### Examine MountainReport.SnowData to workout how to orgnaise and clean data

# pick one element, e.g., the first non-null
sample = df_daily["MountainReport.SnowData"].dropna().iloc[0]

print(sample)
print(type(sample))

[{'Month': 'October', 'Year': 2004, 'Total': 45, 'DaysInMonth': 31, 'Ceiling': 40, 'Days': {'1': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '2': 0, '20': 0, '21': 0, '22': 40, '23': 0, '24': 0, '25': 5, '26': 0, '27': 0, '28': 0, '29': 0, '3': 0, '30': 0, '31': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 'CurrentMonth': True}, {'Month': 'November', 'Year': 2004, 'Total': 75, 'DaysInMonth': 30, 'Ceiling': 35, 'Days': {'1': 8, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 10, '19': 3, '2': 14, '20': 0, '21': 0, '22': 0, '23': 0, '24': 9, '25': 13, '26': 13, '27': 0, '28': 0, '29': 0, '3': 0, '30': 5, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}, 'CurrentMonth': False}, {'Month': 'December', 'Year': 2004, 'Total': 162, 'DaysInMonth': 31, 'Ceiling': 36, 'Days': {'1': 0, '10': 0, '11': 0, '12': 0, '13': 3, '14': 2, '15': 0, '16': 0, '17': 0, '18': 0, '19': 2, '2': 0, '20': 2, '21': 0, '22': 0, '23':

In [26]:
def transform_snowfall(records):
    df = pd.DataFrame(records)
    day_df = df.pop("Days").apply(pd.Series)
    day_df.columns = [f"day_{int(c)}" for c in day_df.columns]
    day_df = day_df.reindex(sorted(day_df.columns, key=lambda x: int(x.split("_")[1])), axis=1)
    return pd.concat([df, day_df], axis=1)

frames = []

for i, snow in df_daily["MountainReport.SnowData"].items():
    # skip missing/empty entries
    if snow is None:
        continue
    if isinstance(snow, float) and math.isnan(snow):
        continue
    if isinstance(snow, (list, tuple, dict)) and len(snow) == 0:
        continue

    try:
        snow_df = transform_snowfall(snow)
        snow_df["source_row"] = i  # optional
        frames.append(snow_df)
    except Exception as exc:
        print(f"Skipped row {i}: {exc}")

pd_daily = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

pd_daily.head(20)

,Month,Year,Total,DaysInMonth,Ceiling,CurrentMonth,day_1,day_2,day_3,day_4,...,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,source_row
0,October,2004,45,31,40,True,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,November,2004,75,30,35,False,8.0,14.0,0.0,0.0,...,0.0,9.0,13.0,13.0,0.0,0.0,0.0,5.0,NaN,0
2,December,2004,162,31,36,False,0.0,0.0,0.0,16.0,...,3.0,0.0,24.0,11.0,0.0,0.0,0.0,3.0,0.0,0
3,January,2005,80,31,35,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0
4,February,2005,6,28,35,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,0
5,March,2005,90,31,35,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,30.0,30.0,10.0,0.0,0.0,0
6,April,2005,29,30,35,False,8.0,11.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
7,October,2005,20,31,35,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,1
8,November,2005,196,30,35,False,10.0,10.0,18.0,0.0,...,0.0,0.0,16.0,20.0,0.0,0.0,12.0,7.0,NaN,1
9,December,2005,107,31,35,False,0.0,28.0,0.0,0.0,...,11.0,0.0,0.0,5.0,7.0,0.0,5.0,18.0,4.0,1
